In [33]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import joblib

In [34]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
336,I too saw this movie when it first came out. I...,positive
969,I voted this a 10 out of 10 simply because it ...,positive
519,"""Most of us at least inhabit two worlds , the ...",positive
809,"I wish ""that '70s show"" would come back on tel...",positive
413,This movie was the worst movie I have seen sin...,negative


In [35]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Asus\AppData\Local\Temp\ipykernel_10856\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [36]:
# df = normalize_text(df)
# df.head()

In [37]:
df['sentiment'].value_counts()

sentiment
negative    251
positive    249
Name: count, dtype: int64

In [38]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [39]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
336,I too saw this movie when it first came out. I...,1
969,I voted this a 10 out of 10 simply because it ...,1
519,"""Most of us at least inhabit two worlds , the ...",1
809,"I wish ""that '70s show"" would come back on tel...",1
413,This movie was the worst movie I have seen sin...,0


In [40]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [41]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [43]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/varun6030/Capstone-Project.mlflow')
dagshub.init(repo_owner='varun6030', repo_name='Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-09-07 19:19:38,439 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/varun6030/Capstone-Project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "varun6030/Capstone-Project"

2025-09-07 19:19:38,452 - INFO - Initialized MLflow to track repo "varun6030/Capstone-Project"


Repository varun6030/Capstone-Project initialized!

2025-09-07 19:19:38,454 - INFO - Repository varun6030/Capstone-Project initialized!


<Experiment: artifact_location='mlflow-artifacts:/38085f9cfdfe488c923afab8d0ea9cf6', creation_time=1757251850439, experiment_id='0', last_update_time=1757251850439, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [46]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving model locally with joblib...")
        model_path = "logreg_model.pkl"
        joblib.dump(model, model_path)

        logging.info("Logging model as an artifact to MLflow/DagsHub...")
        mlflow.log_artifact(model_path, artifact_path="models")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-09-07 19:47:00,273 - INFO - Starting MLflow run...
2025-09-07 19:47:01,463 - INFO - Logging preprocessing parameters...
2025-09-07 19:47:02,884 - INFO - Initializing Logistic Regression model...
2025-09-07 19:47:02,884 - INFO - Fitting the model...
2025-09-07 19:47:02,950 - INFO - Model training complete.
2025-09-07 19:47:02,950 - INFO - Logging model parameters...
2025-09-07 19:47:03,342 - INFO - Making predictions...
2025-09-07 19:47:03,344 - INFO - Calculating evaluation metrics...
2025-09-07 19:47:03,349 - INFO - Logging evaluation metrics...
2025-09-07 19:47:09,008 - INFO - Saving model locally with joblib...
2025-09-07 19:47:09,018 - INFO - Logging model as an artifact to MLflow/DagsHub...
2025-09-07 19:47:10,157 - INFO - Model training and logging completed in 8.69 seconds.
2025-09-07 19:47:10,157 - INFO - Accuracy: 0.648
2025-09-07 19:47:10,160 - INFO - Precision: 0.6716417910447762
2025-09-07 19:47:10,161 - INFO - Recall: 0.6716417910447762
2025-09-07 19:47:10,161 - INFO 

🏃 View run defiant-mole-479 at: https://dagshub.com/varun6030/Capstone-Project.mlflow/#/experiments/0/runs/8b87f2a021d64a529bb7ea77ed7db3d8
🧪 View experiment at: https://dagshub.com/varun6030/Capstone-Project.mlflow/#/experiments/0
